In [13]:
# pip install –U fxcmpy
# pip install ta
# pip install python-socketio

import fxcmpy

import ta

import numpy as np
import pandas as pd

TOKEN = "35b91ea52c03042231d320e88d6ab9dfdc3d0eaa"

api = fxcmpy.fxcmpy(access_token=TOKEN, 
                    log_level='error', server='demo', 
                    log_file='log.txt')




In [6]:
inst = ['EUR/USD', 'USD/JPY', 'GBP/USD', 'USD/CHF', 'EUR/CHF',
        'AUD/USD', 'USD/CAD', 'NZD/USD', 'EUR/GBP', 'EUR/JPY',
        'GBP/JPY', 'CHF/JPY', 'GBP/CHF', 'EUR/AUD', 'EUR/CAD', 
        'AUD/CAD', 'AUD/JPY', 'CAD/JPY', 'NZD/JPY', 'GBP/CAD', 
        'GBP/NZD', 'GBP/AUD', 'AUD/NZD',  'AUD/CHF', 'EUR/NZD',
        'NZD/CHF', 'CAD/CHF', 'NZD/CAD',  'GER30',  'US30']

In [8]:
"""
Created on Wed Jan  1 21:01:11 2020

@author: Ricky Macharm
https
"""

    

def HTFM(inst, p):    
    """To create a binary value of the High Time Frame Momentum indicator where it
        is either going up or down or nowhere.
    """
    df = api.get_candles(inst, period=p, number=200)
    
    # Direction Movement Indcator (DMI)
    df['DMI_neg'] = ta.trend.adx_neg(df.bidhigh, df.bidlow, df.bidclose, n=3, fillna=False)
    df['DMI_pos'] = ta.trend.adx_pos(df.bidhigh, df.bidlow, df.bidclose, n=3, fillna=False)
    df['DMI_Dir'] = df.DMI_pos - df.DMI_neg

    
    # EMA Position
    df['ma10'] = ta.trend.ema_indicator(df.bidclose, n=10, fillna=False)
    df['ma_pos'] = df.bidclose - df.ma10
    
    
    # Momentum Direction
      
    df['mo_Dir'] = np.where((df.ma_pos>0) & (df.DMI_Dir>0), 1,
                           np.where((df.ma_pos<0) & (df.DMI_Dir<0), -1, 0))
    
    return df['mo_Dir'].iloc[-1]



def action(inst,p):    
    """This indicator shows when the entry time frame is in the right mode for entries.
    For short entries, we want the ROC to be overbought and the CCI to be above 0, and 
    for long entries, we want the ROC to be oversold and the CCI to be below 0.
    """
    df = api.get_candles(inst, period=p, number=200)
    
    # Average True Range for stops
    
    df['H-L']=abs(df['bidhigh']-df['bidlow'])
    df['H-PC']=abs(df['bidhigh']-df['bidclose'].shift(1))
    df['L-PC']=abs(df['bidlow']-df['bidclose'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['atr'] = df['TR'].rolling(5).mean()
    

       
    # Two moving averages for trade selection
    df['ma10'] = ta.trend.ema_indicator(df.bidclose, n=10, fillna=False)
    df['ma25'] = ta.trend.ema_indicator(df.bidclose, n = 25, fillna = False)
    df['ma_diff'] = df.ma10 - df.ma25


    # Rate of Change overbought/oversold for preparation for entries
    df['roc'] = df.bidclose - df.bidclose.shift(1)
    
    df['mo_pinball'] = ta.momentum.rsi(df['roc'], n=3, fillna = False)
    
    df['cci'] = ta.trend.cci(df.bidhigh, df.bidlow, df.bidclose, n=6, c=0.015, fillna=False)
    
        
    df['Position'] = np.where(((df.mo_pinball<30) & ( df.cci<0) & (df['ma_diff'] > 0)),'BUY',
                           np.where(((df.mo_pinball>70) & (df.cci> 0) & (df['ma_diff'] < 0)), 'SELL', 'nowhere'))
    
    
    # Entry
    def entry(df):
        if df.Position == 'BUY': return (df.bidclose + df.atr/2 )
        elif df.Position == 'SELL': return (df.bidclose - df.atr/2)
        else: return 0.0
   
    df['ENTRY price'] = df.apply(entry, axis=1)
    
    # Stops
    def stops(df):
        if df.Position == 'BUY': return (df['ENTRY price'] - (df.atr * 2.5))
        elif df.Position == 'SELL': return (df['ENTRY price'] + (df.atr * 2.5))
        else: return 0.0

    df['STOP price'] = df.apply(stops, axis=1)

    #Take Profit
    def take_profit(df):
        if df.Position == 'BUY': return (df['ENTRY price'] + (5 * df.atr))
        elif df.Position == 'SELL': return (df['ENTRY price'] - (5 * df.atr))
        else: return 0.0
    
    
    df['TAKE Profit'] = df.apply(take_profit, axis=1)
    df['Time Frame'] = p
    df['Instrument']=inst
    df['Close price'] = df.bidclose
    df['ATR pips'] = df['atr'] 
    
    
    return df[['Instrument', 'Time Frame','Position', 'Close price', 'ATR pips', 'ENTRY price', 'STOP price', 'TAKE Profit']]



def trades(inst, p1, p2, p3):
    if ((HTFM(inst, p1) == 1) &(HTFM(inst, p2) == 1) & (action(inst,p3)['Position'].iloc[-1] == 'BUY') | 
       (HTFM(inst, p1) == -1) & (HTFM(inst, p2) == -1) & (action(inst,p3)['Position'].iloc[-1] == 'SELL')):
        return print(action(inst, p3).iloc[-1])
    
    else:  pass 


def dashboard(p1, p2, p3):
    
    for i in range(0, len(inst)):
        if trades(inst[i], p1, p2, p3) == None: pass
        else:
                     
            print(f'{trades(inst[i],p1, p2, p3)} ')
               
    return print('\nApply due diligence. \nEntries and stops around support or resistance. \nWatch on Demo!')

In [9]:
dashboard('M1', 'W1', 'D1')


Apply due diligence. 
Entries and stops around support or resistance. 
Watch on Demo!


In [10]:
dashboard('W1', 'D1', 'H4')

Instrument     AUD/JPY
Time Frame          H4
Position           BUY
Close price     75.783
ATR pips         0.236
ENTRY price     75.901
STOP price      75.311
TAKE Profit     77.081
Name: 2020-01-17 18:00:00, dtype: object

Apply due diligence. 
Entries and stops around support or resistance. 
Watch on Demo!


In [11]:
dashboard('D1', 'H4', 'H1')


Apply due diligence. 
Entries and stops around support or resistance. 
Watch on Demo!
